In [1]:
from fastai.imports import *
from lemonpie.basics import *
from lemonpie.preprocessing import vocab
from lemonpie.preprocessing.transform import *
from lemonpie.data import *
from lemonpie import models

import torch
import pytorch_lightning as pl

In [2]:
COHERENT_DATA_STORE = '/home/vinod/code/datasets/coherent'
COHERENT_DATAGEN_DATE = '08-10-2021'
COHERENT_CONDITIONS = {
    "heart_failure" : "88805009",
    "coronary_heart" : "53741008",
    "myocardial_infarction" : "22298006",
    "stroke" : "230690007",
    "cardiac_arrest" : "410429000"
}

In [3]:
COHERENT_LABELS = list(COHERENT_CONDITIONS.keys())
COHERENT_LABELS

['heart_failure',
 'coronary_heart',
 'myocardial_infarction',
 'stroke',
 'cardiac_arrest']

# Get data

In [4]:
coherent_data = MultimodalEHRData(
    COHERENT_DATA_STORE, 
    COHERENT_LABELS,     
    age_start=240,
    age_range=120,
    start_is_date=False,
    age_in_months=True, 
    lazy_load_gpu=True)

In [5]:
demograph_dims, rec_dims, demograph_dims_wd, rec_dims_wd = vocab.get_all_emb_dims(vocab.EhrVocabList.load(COHERENT_DATA_STORE))
dls, pos_wts = coherent_data.get_data()

In [6]:
coherent_data.modality_types

{'train': ['0', '21', '30', '31', '11', '1', '20', '10'],
 'valid': ['21', '30', '31', '11', '1', '20', '10'],
 'test': ['21', '30', '31', '11', '1', '20', '10']}

In [7]:
dls

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f3b9eb8de80>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x7f3b9eb92190>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f3b9eba0070>}

In [8]:
train_dl = dls["train"]
valid_dl = dls["valid"]
test_dl = dls["test"]

# Unimodal Models - One Per Modality
These are stubs / dummys to be replaced by the real ones.

In [141]:
class UnimodalModel(pl.LightningModule):
    def __init__(self, input_dims: tuple):
        super().__init__()

        # args
        self.input_dims = input_dims

        # model
        self.model = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 30),
            nn.ReLU(),
            nn.Linear(30, 5),
            nn.Sigmoid()
            )
        
    
    def forward(self, x):
        assert x[0].size() == self.input_dims
        bs = len(x)
        fake_x = torch.randn((bs, 10), device=self.device)
        # return self.model(fake_x)
        out = self.model(fake_x)
        print(out.device, out.shape)
        return out

    def training_step(self, *args, **kwargs):
        return super().training_step(*args, **kwargs)

    def configure_optimizers(self):
        return super().configure_optimizers()



## MRI

In [142]:
mri_model = UnimodalModel((4,4))
mri_model(torch.randn(3,4,4)) # batch_size=3

cpu torch.Size([3, 5])


tensor([[0.4491, 0.4612, 0.5426, 0.4824, 0.5076],
        [0.4563, 0.4703, 0.5232, 0.4784, 0.4925],
        [0.4457, 0.4331, 0.5131, 0.4999, 0.4860]], grad_fn=<SigmoidBackward0>)

In [143]:
trainer = pl.Trainer(max_epochs=5, accelerator='gpu', devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [144]:
trainer.fit(mri_model, train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/core/module.py:1368: UserWarning: `configure_optimizers` must be implemented to be used with the Lightning Trainer
  rank_zero_warn("`configure_optimizers` must be implemented to be used with the Lightning Trainer")
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/core/optimizer.py:182: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.0 K 
-------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/21 [00:00<?, ?it/s] 

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1891: PossibleUserWarning: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  10%|▉         | 2/21 [00:01<00:14,  1.28it/s, loss=nan, v_num=11]

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/core/module.py:670: UserWarning: `training_step` must be implemented to be used with the Lightning Trainer
  rank_zero_warn("`training_step` must be implemented to be used with the Lightning Trainer")


Epoch 4: 100%|██████████| 21/21 [00:03<00:00,  5.77it/s, loss=nan, v_num=11]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 21/21 [00:03<00:00,  5.75it/s, loss=nan, v_num=11]


In [145]:
trainer.save_checkpoint("./mri_model_pth")

## DNA

In [10]:
dna_model = UnimodalModel((3,2))
dna_model(torch.randn(2,3,2))

tensor([[0.5268, 0.4636, 0.5170, 0.5797, 0.5368],
        [0.5228, 0.5087, 0.4918, 0.5637, 0.5334]], grad_fn=<SigmoidBackward0>)

## ECG

In [11]:
ecg_model = UnimodalModel((5,))
ecg_model(torch.randn((4,5,)))

tensor([[0.5476, 0.5253, 0.4565, 0.4424, 0.5881],
        [0.5092, 0.4856, 0.4894, 0.4981, 0.5547],
        [0.5469, 0.5157, 0.4929, 0.4563, 0.5946],
        [0.4943, 0.5055, 0.4931, 0.4860, 0.5348]], grad_fn=<SigmoidBackward0>)

## EHR

In [115]:
ehr_model = models.EHR_LSTM(
    demograph_dims,
    rec_dims,
    demograph_dims_wd,
    rec_dims_wd,
    len(COHERENT_LABELS),
    pos_wts["train"], 
    pos_wts["valid"],
    optim="adam",
    base_lr=0.001,
)
ehr_trainer = pl.Trainer(precision=16, accelerator='gpu', devices=-1, max_epochs=5)
# ehr_output = ehr_trainer.test(ehr_model, test_dl)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
ehr_trainer.fit(ehr_model, train_dl, valid_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | train_loss_fn | BCEWithLogitsLoss | 0     
1 | valid_loss_fn | BCEWithLogitsLoss | 0     
2 | embs          | ModuleList        | 12.0 K
3 | embgs         | ModuleList        | 19.5 K
4 | input_dp      | InputDropout      | 0     
5 | lstm          | LSTM              | 250 K 
6 | linear        | Sequential        | 7.4 M 
7 | train_metrics | MetricCollection  | 0     
8 | valid_metrics | MetricCollection  | 0     
9 | test_metrics  | MetricCollection  | 0     
----------------------------------------------------
7.7 M     Trainable params
0         Non-trainable params
7.7 M     Total params
15.320    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/28 [00:00<?, ?it/s]                            

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 13. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1891: PossibleUserWarning: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8

Epoch 0:   4%|▎         | 1/28 [00:03<01:31,  3.39s/it, loss=0.996, v_num=9]

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 59. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


...

Epoch 0: 100%|██████████| 28/28 [00:08<00:00,  3.12it/s, loss=1.58, v_num=9]

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 14. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 12. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 34. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightni

Epoch 4: 100%|██████████| 28/28 [00:08<00:00,  3.17it/s, loss=0.99, v_num=9] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 28/28 [00:09<00:00,  3.10it/s, loss=0.99, v_num=9]


- [Lightning warning details](https://github.com/Lightning-AI/lightning/issues/10349#issuecomment-961340903)

In [15]:
ehr_trainer.save_checkpoint("./ehr_model_pth")

In [104]:
for batch in train_dl:
    pts, ys, ms, other = batch
    print(len(ms))

64
59
64
64
64
2
64
64
64
64
64
64
13
10
52
13
64
64
64
19
22


# Unimodal Datasets

```python
class MRIDataset(torch.utils.data.Dataset):
    def __init__(self, datastore: str, tensor_sz: tuple):
        super().__init__()
        self.mri_dir = f"{datastore}/output/dicom"
        self.tensor_sz = tensor_sz
    
    def __getitem__(self, i):
        mri_fname = glob.glob(f"{self.mri_dir}/*{i}*")
        if len(mri_fname) == 1:
            return torch.full(self.tensor_sz, 1)
        else:
            raise Exception(f"MRI filename match error - found {len(mri_fname)} files with ptid: {i}.")

    def __len__(self):
        return 1
```

```python
class DNADataset(torch.utils.data.Dataset):
    def __init__(self, datastore: str, tensor_sz: tuple):
        super().__init__()
        self.dna_dir = f"{datastore}/output/dna"
        self.tensor_sz = tensor_sz
    
    def __getitem__(self, i):
        dna_fname = glob.glob(f"{self.dna_dir}/*{i}*")
        if len(dna_fname) == 1:
            return torch.full(self.tensor_sz, 10)
        else:
            raise Exception(f"DNA filename match error - found {len(dna_fname)} files with ptid: {i}.")

    def __len__(self):
        return 1
```

```python
class ECGDataset(torch.utils.data.Dataset):
    def __init__(self, datastore: str, tensor_sz: tuple):
        super().__init__()
        ecg_data = pd.read_csv(f"{datastore}/ecg.csv")
        self.ecg_pids = ecg_data.patient.unique()
        self.tensor_sz = tensor_sz
    
    def __getitem__(self, i):
        
        if i in self.ecg_pids:
            return torch.full(self.tensor_sz, 20)
        else:
            raise Exception(f"ptid: {i} - not found in ECG data.")

    def __len__(self):
        return 1
```

# Late Fusion

| Modality Type | Modalities            |   
|---	        |---	                |
| **0**	        | **EHR**               |
| **1**         | EHR + **MRI**         |
| **10**        | EHR + **DNA**         |      
| 11   	        | EHR + MRI + DNA       |
| **20**        | EHR + **ECG**         |
| 21            | EHR + MRI + ECG       |
| 30            | EHR + DNA + ECG       |
| 31            | EHR + MRI + DNA + ECG |


In [16]:
batch = next(iter(valid_dl))   
batch

[[ptid:1a82483d-7eb2-d5e0-1e1f-398ba129b18b, birthdate:1936-12-22, [('heart_failure', True), ('coronary_heart', False)].., device:cpu,
  ptid:6dc8bd6b-e2a8-92bf-613d-8b477eb87d7c, birthdate:1911-12-23, [('heart_failure', True), ('coronary_heart', False)].., device:cpu,
  ptid:844a37ff-ce26-6338-fd6a-0bc1e925a702, birthdate:1933-03-15, [('heart_failure', True), ('coronary_heart', False)].., device:cpu],
 tensor([[1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.]]),
 [21, 21, 21],
 [tensor([[[1, 1, 1, 1],
           [1, 1, 1, 1],
           [1, 1, 1, 1],
           [1, 1, 1, 1]],
  
          [[1, 1, 1, 1],
           [1, 1, 1, 1],
           [1, 1, 1, 1],
           [1, 1, 1, 1]],
  
          [[1, 1, 1, 1],
           [1, 1, 1, 1],
           [1, 1, 1, 1],
           [1, 1, 1, 1]]]),
  tensor([[20, 20, 20, 20, 20],
          [20, 20, 20, 20, 20],
          [20, 20, 20, 20, 20]])]]

In [17]:
pts, ys, mod_type, other = batch

In [18]:
torch.sigmoid(ehr_model(pts))

tensor([[0.5317, 0.5461, 0.4880, 0.4909, 0.4987],
        [0.5642, 0.5709, 0.4845, 0.5405, 0.4524],
        [0.5330, 0.4908, 0.3036, 0.4859, 0.5562]], grad_fn=<SigmoidBackward0>)

In [19]:
mri_input, ecg_input = other

In [20]:
mri_model(mri_input)

NameError: name 'mri_model' is not defined

In [ ]:
ecg_model(ecg_input)

tensor([[0.4668, 0.4984, 0.5840, 0.4353, 0.3662],
        [0.5483, 0.4521, 0.5888, 0.5441, 0.3307],
        [0.5181, 0.3867, 0.4538, 0.5974, 0.3182]], grad_fn=<SigmoidBackward0>)

In [21]:
def run_late_fusion(dl, verbose=False):

    all_batches = []
    total_length = 0

    for batch in dl:

        batch_output = []
        pts, ys, mod_type, other = batch
        
        total_length += len(pts)
        if verbose:
            print(f"modality: {mod_type[0]}, length: {len(pts)}, first ptid: {pts[0].ptid}")

        batch_output.append(torch.sigmoid(ehr_model(pts)))

        if mod_type[0] == 1:
            mri_input = other 
            batch_output.append(mri_model(mri_input))

        elif mod_type[0] == 10:
            dna_input = other 
            batch_output.append(dna_model(dna_input))
            
        elif mod_type[0] == 11:
            mri_input, dna_input = other 
            batch_output.append(mri_model(mri_input))
            batch_output.append(dna_model(dna_input))

        elif mod_type[0] == 20:
            ecg_input = other 
            batch_output.append(ecg_model(ecg_input))

        elif mod_type[0] == 21:
            mri_input, ecg_input = other 
            batch_output.append(mri_model(mri_input))
            batch_output.append(ecg_model(ecg_input))

        elif mod_type[0] == 30:
            dna_input, ecg_input = other 
            batch_output.append(dna_model(dna_input))
            batch_output.append(ecg_model(ecg_input))

        elif mod_type[0] == 31:
            mri_input, dna_input, ecg_input = other
            batch_output.append(mri_model(mri_input))
            batch_output.append(dna_model(dna_input))
            batch_output.append(ecg_model(ecg_input))

        # avg across multimodal models for every patient in batch
        avgd = torch.mean(torch.stack(batch_output), dim=0)         
        all_batches.append(avgd)


    print(f"Completed {total_length} patients.")

    # flatten across batches - list of lists (batches) to flat
    output = [pt for batch in all_batches for pt in batch]
    return output

In [22]:
test_result = run_late_fusion(test_dl)

NameError: name 'mri_model' is not defined

In [22]:
valid_result = run_late_fusion(valid_dl, verbose=True)

modality: 21, length: 3, first ptid: 1a82483d-7eb2-d5e0-1e1f-398ba129b18b
modality: 30, length: 13, first ptid: 2e1cf98c-70ce-4f8f-36da-b2eef4960ecc
modality: 31, length: 2, first ptid: 4a1a224f-54d6-66f1-3755-4c8489f2a5de
modality: 11, length: 14, first ptid: aead835e-66f2-d3b9-099c-c33844a70748
modality: 1, length: 12, first ptid: 972f6a59-3921-36bc-64bb-253d6241748b
modality: 20, length: 34, first ptid: e1875ec6-e10f-c9d8-d388-fd3abfbc4a87
modality: 10, length: 50, first ptid: 1e73e6da-68f0-0ffc-4062-a4647b5b67c4
Completed 128 patients.


In [23]:
train_result = run_late_fusion(train_dl)

Completed 1022 patients.


In [24]:
assert len(test_result) == len(valid_result) == 128
assert len(train_result) == 1022

# Joint Fusion

In [48]:
pts, y, m, other = next(iter(valid_dl))

In [49]:
pts

[ptid:1a82483d-7eb2-d5e0-1e1f-398ba129b18b, birthdate:1936-12-22, [('heart_failure', True), ('coronary_heart', False)].., device:cpu,
 ptid:6dc8bd6b-e2a8-92bf-613d-8b477eb87d7c, birthdate:1911-12-23, [('heart_failure', True), ('coronary_heart', False)].., device:cpu,
 ptid:844a37ff-ce26-6338-fd6a-0bc1e925a702, birthdate:1933-03-15, [('heart_failure', True), ('coronary_heart', False)].., device:cpu]

In [66]:
m[0]

21

## MRI

In [52]:
mri_model = UnimodalModel.load_from_checkpoint("./mri_model_pth", input_dims=(4,4))

In [53]:
mri_layers_trunc = list(mri_model.model.children())[:-2]

In [54]:
mri_model.model = nn.Sequential(*mri_layers_trunc)

In [55]:
# mri_model

In [56]:
mri_repr = mri_model(mri_input)

In [57]:
mri_repr.shape

torch.Size([3, 30])

## EHR

In [58]:
ehr_model = models.EHR_LSTM.load_from_checkpoint(
    "./ehr_model_pth",
    demograph_dims=demograph_dims,
    rec_dims=rec_dims,
    demograph_dims_wd=demograph_dims_wd,
    rec_dims_wd=rec_dims_wd,
    labels=len(COHERENT_LABELS),
    train_pos_wts=pos_wts["train"], 
    valid_pos_wts=pos_wts["valid"],
    optim="adam",
    base_lr=0.001,
)

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [59]:
ehr_layers_trunc = list(ehr_model.linear.children())[:-2]

In [60]:
ehr_model.linear = nn.Sequential(*ehr_layers_trunc)

In [61]:
# ehr_model

In [62]:
ehr_repr = ehr_model(pts)

In [63]:
ehr_repr.shape

torch.Size([3, 3328])

## Joint Model

In [64]:
concated = torch.concat((ehr_repr, mri_repr), dim=1)

In [65]:
concated.shape

torch.Size([3, 3358])

In [175]:
class JointFusion(pl.LightningModule):
    def __init__(self,
            demograph_dims,
            rec_dims,
            demograph_wd,
            rec_wd,
            num_labels,
            train_pos_wts,
            valid_pos_wts
            ):

        super().__init__()

        ## args
        self.train_loss_fn = nn.BCEWithLogitsLoss(pos_weight=train_pos_wts)
        self.valid_loss_fn = nn.BCEWithLogitsLoss(pos_weight=valid_pos_wts)


        # ehr
        # ehr_repr = ehr_model.lin_o.in_features
        self.ehr_model = models.EHR_LSTM.load_from_checkpoint(
            "./ehr_model_pth",
            demograph_dims=demograph_dims,
            rec_dims=rec_dims,
            demograph_dims_wd=demograph_dims_wd,
            rec_dims_wd=rec_dims_wd,
            labels=len(COHERENT_LABELS),
            train_pos_wts=pos_wts["train"], 
            valid_pos_wts=pos_wts["valid"],
            optim="adam",
            base_lr=0.001,
        )
        ehr_layers_trunc = list(self.ehr_model.linear.children())[:-2]
        self.ehr_model.linear = nn.Sequential(*ehr_layers_trunc)

        # mri
        self.mri_model = UnimodalModel.load_from_checkpoint("./mri_model_pth", input_dims=(4,4))
        mri_layers_trunc = list(self.mri_model.model.children())[:-2]
        self.mri_model.model = nn.Sequential(*mri_layers_trunc)
        
        # self.dna_model = dna_model
        # self.ecg_model = ecg_model

        # model
        self.model = nn.Sequential(nn.Linear(3358, 4000),
                      nn.ReLU(),
                      nn.Linear(4000, 5000),
                      nn.ReLU(),
                      nn.Linear(5000, 5),
                      nn.Sigmoid())
        
    
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        pts, ys, ms, other = batch
        xb = [pt.to_gpu(non_block=True) for pt in pts]
        yb = ys
        # yb = ys.to(self.device, non_blocking=True)
        ehr_repr = self.ehr_model(pts)
        
        # m conditions
        if m[0] == 1:
            mri_input = other 

        elif m[0] == 10:
            dna_input = other 
            
        elif m[0] == 11:
            mri_input, dna_input = other 

        elif m[0] == 20:
            ecg_input = other 

        elif m[0] == 21:
            mri_input, ecg_input = other 
            batch_output.append(mri_model(mri_input))
            batch_output.append(ecg_model(ecg_input))

        elif m[0] == 30:
            dna_input, ecg_input = other 
            batch_output.append(dna_model(dna_input))
            batch_output.append(ecg_model(ecg_input))

        elif m[0] == 31:
            mri_input, dna_input, ecg_input = other
            batch_output.append(mri_model(mri_input))
            batch_output.append(dna_model(dna_input))
            batch_output.append(ecg_model(ecg_input))
        mri_input, ecg_input = other
        
        # send through other modalities
        # mri_input.to(self.device, non_blocking=True)
        mri_repr = self.mri_model(mri_input)

        # concat reprs
        concated_reprs = torch.concat((ehr_repr, mri_repr), dim=1)

        # send through self
        # concated_reprs.to(self.device, non_blocking=True)
        y_hat = self(concated_reprs)
        train_loss = self.train_loss_fn(y_hat, yb)

        self.log("train_loss", train_loss, on_step=True, on_epoch=True)
        # self.train_metrics.update(y_hat, yb.int())
        # self.log_dict(self.train_metrics.compute(), on_step=False, on_epoch=True)

        return train_loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer



In [176]:
fusion_model = JointFusion(
    demograph_dims,
    rec_dims,
    demograph_dims_wd,
    rec_dims_wd,
    len(COHERENT_LABELS),
    pos_wts["train"], 
    pos_wts["valid"])

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [177]:
fusion_trainer = pl.Trainer(precision=16, accelerator='gpu', devices=-1, max_epochs=5)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [178]:
fusion_trainer.fit(fusion_model, train_dataloaders=valid_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | train_loss_fn | BCEWithLogitsLoss | 0     
1 | valid_loss_fn | BCEWithLogitsLoss | 0     
2 | ehr_model     | EHR_LSTM          | 7.6 M 
3 | mri_model     | UnimodalModel     | 850   
4 | model         | Sequential        | 33.5 M
----------------------------------------------------
41.1 M    Trainable params
0         Non-trainable params
41.1 M    Total params
82.220    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/7 [00:00<?, ?it/s] 

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1891: PossibleUserWarning: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


torch.Size([3, 4, 4])
cuda:0 torch.Size([3, 30])
Epoch 0:  14%|█▍        | 1/7 [00:01<00:09,  1.59s/it, loss=1.06, v_num=12]torch.Size([13, 3, 2])


AssertionError: 